In [ ]:
from RAGLibrary import myWidgets, myRAG, checkConstruct, createSchema, faissConvert, embedding
import os
import json
import torch
import faiss
import pickle
import logging
import logging
import numpy as np
from typing import Any, Dict, List, Tuple

In [ ]:
widgets_list = myWidgets.create_name_form()

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
force_download = True

In [ ]:
""" DEFINE """

data   = widgets_list[0] #HBox 1
keys   = widgets_list[1] #HBox 2
choose = widgets_list[2] #HBox 3

embedd_model = widgets_list[3]
search_egine = widgets_list[4]
rerank_model = widgets_list[5]
respon_model = widgets_list[6]
API_drop     = widgets_list[7]
button_box   = widgets_list[8]

# HBox 1
file_name = data.children[0]
file_type = data.children[1]

# HBox 2
data_key = keys.children[0]
embe_key = keys.children[1]

# HBox 3
switch_model = choose.children[0]
merge_otp    = choose.children[1]
path_end_val = choose.children[1]

# Get value
data_folder   = file_name.value
file_type_val = file_type.value

data_key_val  = data_key.value
embe_key_val  = embe_key.value

API_key_val = API_drop.value
switch      = switch_model.value
merge       = merge_otp.value
path_end    = path_end_val.value

embedding_model = embedd_model.value
searching_egine = search_egine.value
reranking_model = rerank_model.value
responing_model = respon_model.value


# Define
base_path = f"../Data/{data_folder}/{file_type_val}_{data_folder}"

json_file_path = f"{base_path}_Database.json"
schema_ex_path = f"{base_path}_Schema.json"
embedding_path = f"{base_path}_Embeds_{merge}"

torch_path  = f"{embedding_path}.pt"
faiss_path  = f"{embedding_path}.faiss"
mapping_path = f"{embedding_path}_mapping.json"
mapping_data = f"{embedding_path}_map_data.json"

FILE_TYPE    = file_type_val
DATA_KEY     = data_key_val
EMBE_KEY     = embe_key_val
SWITCH       = switch
EMBEDD_MODEL = embedding_model
SEARCH_EGINE = searching_egine
RERANK_MODEL = reranking_model
RESPON_MODEL = responing_model

if FILE_TYPE == "Data":
    MERGE = merge
else: 
    MERGE = "no_Merge"

API_KEY = API_key_val

SEARCH_ENGINE = faiss.IndexFlatIP

print("\n")
print(f"Embedder: {EMBEDD_MODEL}")
print(f"Searcher: {SEARCH_EGINE}")
print(f"Reranker: {RERANK_MODEL}")
print(f"Responer: {RESPON_MODEL}")
print(f"Data Key: {DATA_KEY}")
print(f"Embe Key: {EMBE_KEY}")
print(f"Database: {json_file_path}")
print(f"Torch   : {torch_path}")
print(f"Faiss   : {faiss_path}")
print(f"Mapping : {mapping_path}")
print(f"Map Data: {mapping_data}")
print(f"Schema  : {schema_ex_path}")
print(f"Model   : {SWITCH}")
print(f"Merge   : {MERGE}")
print(f"API Key : {API_KEY}")

In [ ]:
""" CHECK EMBEDDDING CONTRUCTION """

def print_json(pt_path: str) -> None:
    try:
        if not os.path.exists(pt_path):
            print(f"File không tồn tại: {pt_path}")
            return

        data = torch.load(pt_path, map_location="cpu", weights_only=False)

        if isinstance(data, dict) and f"{DATA_KEY}" in data:
            content = data[f"{DATA_KEY}"]
        else:
            print(f"Dữ liệu không đúng định dạng: không tìm thấy key {DATA_KEY}")
            return

        if not isinstance(content, list) or not content:
            print("Dữ liệu rỗng hoặc không phải danh sách")
            return

        first_json = content[0]

        def process_json(obj: any) -> any:
            if isinstance(obj, dict):
                return {k: process_json(v) for k, v in obj.items()}
            elif isinstance(obj, list) and all(isinstance(x, (float, int)) for x in obj):
                return len(obj)
            elif isinstance(obj, list):
                return [process_json(item) for item in obj]
            return obj

        processed_json = process_json(first_json)

        print(json.dumps(processed_json, ensure_ascii=False, indent=2))

    except Exception as e:
        print(f"Lỗi khi đọc file .pt: {str(e)}")

print_json(torch_path)

In [ ]:
# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def inspect_torch_path(torch_path: str) -> None:
    """
    Kiểm tra nội dung file .pt để xác định cấu trúc và dữ liệu.
    
    Args:
        torch_path: Đường dẫn đến file .pt (torch_path từ DEFINE)
    """
    try:
        logging.info(f"Đang tải file .pt: {torch_path}")
        data = torch.load(torch_path, map_location=torch.device('cpu'), weights_only=False)
        
        logging.info(f"Kiểu dữ liệu: {type(data)}")
        if isinstance(data, dict):
            logging.info(f"Số lượng khóa cấp cao nhất: {len(data)}")
            for i, (key, value) in enumerate(data.items()):
                logging.info(f"Khóa: {key}, Kiểu giá trị: {type(value)}, Giá trị mẫu: {str(value)[:100]}...")
                if i >= 5:
                    break
        elif isinstance(data, list):
            logging.info(f"Số lượng phần tử: {len(data)}")
            for i, value in enumerate(data[:5]):
                logging.info(f"Phần tử {i}, Kiểu giá trị: {type(value)}, Giá trị mẫu: {str(value)[:100]}...")
        else:
            logging.info(f"Dữ liệu: {str(data)[:100]}...")
    except Exception as e:
        logging.error(f"Lỗi khi tải file .pt: {str(e)}")
        raise

In [ ]:
def extract_embeddings_and_data(data: Any, prefix: str = "") -> Tuple[List[Tuple[str, np.ndarray]], Dict[str, Any]]:
    """
    Trích xuất đệ quy embedding và dữ liệu thông thường từ dữ liệu đầu vào.
    Tìm embedding dựa trên khóa chứa 'embedding' (như contents.<i>.Merged_embedding).
    
    Args:
        data: Dữ liệu đầu vào (từ điển, danh sách, v.v.)
        prefix: Tiền tố cho khóa
    """
    embeddings_list = []
    data_mapping = {}
    
    if isinstance(data, dict):
        for key, value in data.items():
            full_key = f"{prefix}.{key}" if prefix else key
            if isinstance(value, dict):
                sub_embeds, sub_data = extract_embeddings_and_data(value, full_key)
                embeddings_list.extend(sub_embeds)
                data_mapping.update(sub_data)
            elif isinstance(value, list) and value and isinstance(value[0], dict):
                for i, item in enumerate(value):
                    sub_embeds, sub_data = extract_embeddings_and_data(item, f"{full_key}.{i}")
                    embeddings_list.extend(sub_embeds)
                    data_mapping.update(sub_data)
            elif isinstance(value, (torch.Tensor, np.ndarray)):
                try:
                    embedding = value.cpu().numpy() if isinstance(value, torch.Tensor) else value
                    if embedding.ndim > 1:
                        embedding = embedding.flatten()
                    embeddings_list.append((full_key, embedding))
                except Exception as e:
                    logging.warning(f"Lỗi khi xử lý embedding tại {full_key}: {str(e)}")
                    data_mapping[full_key] = value
            elif isinstance(value, (list, tuple)) and full_key.lower().find("embedding") != -1:
                try:
                    embedding = np.array(value, dtype=np.float32)
                    if embedding.ndim > 1:
                        embedding = embedding.flatten()
                    embeddings_list.append((full_key, embedding))
                except Exception as e:
                    logging.warning(f"Lỗi khi chuyển danh sách thành embedding tại {full_key}: {str(e)}")
                    data_mapping[full_key] = value
            else:
                data_mapping[full_key] = value
    
    elif isinstance(data, list):
        for i, item in enumerate(data):
            full_key = f"{prefix}.item{i}" if prefix else f"item{i}"
            if isinstance(item, (dict, list)):
                sub_embeds, sub_data = extract_embeddings_and_data(item, full_key)
                embeddings_list.extend(sub_embeds)
                data_mapping.update(sub_data)
            elif isinstance(item, (torch.Tensor, np.ndarray)):
                try:
                    embedding = item.cpu().numpy() if isinstance(item, torch.Tensor) else item
                    if embedding.ndim > 1:
                        embedding = embedding.flatten()
                    embeddings_list.append((full_key, embedding))
                except Exception as e:
                    logging.warning(f"Lỗi khi xử lý embedding tại {full_key}: {str(e)}")
                    data_mapping[full_key] = item
            elif isinstance(item, (list, tuple)) and "embedding" in prefix.lower():
                try:
                    embedding = np.array(item, dtype=np.float32)
                    if embedding.ndim > 1:
                        embedding = embedding.flatten()
                    embeddings_list.append((full_key, embedding))
                except Exception as e:
                    logging.warning(f"Lỗi khi chuyển danh sách thành embedding tại {full_key}: {str(e)}")
                    data_mapping[full_key] = item
            else:
                data_mapping[full_key] = item
    
    return embeddings_list, data_mapping

In [ ]:
def create_faiss_index(embeddings: List[Tuple[str, np.ndarray]], nlist: int = 100) -> Tuple[faiss.Index, Dict[str, int]]:
    """
    Tạo chỉ mục FAISS (IndexFlatIP) từ danh sách (khóa, embedding).
    
    Args:
        embeddings: Danh sách các cặp (khóa, embedding)
        nlist: Số lượng cụm cho IndexFlatIP
    """
    if not embeddings:
        raise ValueError("Không tìm thấy embedding trong dữ liệu đầu vào. Vui lòng kiểm tra file .pt.")
    
    embedding_dim = len(embeddings[0][1])
    if not all(len(emb) == embedding_dim for _, emb in embeddings):
        raise ValueError("Tất cả embedding phải có cùng chiều.")
    
    embedding_matrix = np.array([emb for _, emb in embeddings]).astype('float32')    
    logging.info("Đang thêm embedding vào chỉ mục...")

    embedding_dim = embedding_matrix.shape[1]
    
    # Tạo IndexFlatIP
    index = faiss.IndexFlatIP(embedding_dim)
    index.add(embedding_matrix)
    
    key_to_index = {key: idx for idx, (key, _) in enumerate(embeddings)}
    
    return index, key_to_index

In [ ]:
def convert_pt_to_faiss(torch_path: str, faiss_path: str, mapping_path: str, data_path: str, data_key: str, nlist: int = 100, use_pickle: bool = False) -> None:

    """
    Chuyển file .pt sang chỉ mục FAISS và lưu ánh xạ khóa cùng dữ liệu thông thường.
    Sử dụng torch_path (torch_path), faiss_path, mapping_path, data_path từ DEFINE.
    
    Args:
        torch_path: Đường dẫn đến file .pt (torch_path)
        faiss_path: Đường dẫn lưu chỉ mục FAISS
        mapping_path: Đường dẫn lưu ánh xạ khóa
        data_path: Đường dẫn lưu dữ liệu thông thường
        use_pickle: Nếu True, lưu dưới dạng pickle thay vì JSON
        nlist: Số lượng cụm cho IndexFlatIP
    """
    try:
        # Kiểm tra file .pt tồn tại
        if not os.path.exists(torch_path):
            raise FileNotFoundError(f"File .pt không tồn tại: {torch_path}")
        
        # Tạo thư mục đầu ra nếu chưa tồn tại
        os.makedirs(os.path.dirname(faiss_path), exist_ok=True)
        
        # Kiểm tra cấu trúc file .pt
        inspect_torch_path(torch_path)
        
        # Tải file .pt
        logging.info(f"Đang tải file .pt: {torch_path}")
        data = torch.load(torch_path, map_location=torch.device('cpu'), weights_only=False)
        
        # Trích xuất embedding và dữ liệu thông thường
        logging.info("Đang trích xuất embedding và dữ liệu...")
        embeddings_list, data_mapping = extract_embeddings_and_data(data)
        
        # Kiểm tra xem có embedding nào không
        if not embeddings_list:
            logging.error("Không tìm thấy embedding nào trong file .pt. Vui lòng kiểm tra cấu trúc dữ liệu.")
            raise ValueError("Không tìm thấy embedding nào trong file .pt.")
        
        logging.info(f"Tìm thấy {len(embeddings_list)} embedding.")
        
        # Tạo chỉ mục FAISS
        logging.info("Đang tạo chỉ mục FAISS...")
        faiss_index, key_to_index = create_faiss_index(embeddings_list, nlist=nlist)
        
        # Lưu chỉ mục FAISS
        logging.info(f"Đang lưu chỉ mục FAISS vào {faiss_path}")
        faiss.write_index(faiss_index, faiss_path)
        
        # Lưu ánh xạ khóa sang chỉ số
        logging.info(f"Đang lưu ánh xạ khóa vào {mapping_path}")
        if use_pickle:
            with open(mapping_path, 'wb') as f:
                pickle.dump(key_to_index, f)
        else:
            with open(mapping_path, 'w', encoding='utf-8') as f:
                json.dump(key_to_index, f, indent=4, ensure_ascii=False)
        
        # Lưu dữ liệu thông thường
        logging.info(f"Đang lưu dữ liệu thông thường vào {data_path}")
        if use_pickle:
            with open(data_path, 'wb') as f:
                pickle.dump(data_mapping, f)
        else:
            with open(data_path, 'w', encoding='utf-8') as f:
                json.dump(data_mapping, f, indent=4, ensure_ascii=False)
        
        logging.info("Chuyển đổi hoàn tất.")
        
    except Exception as e:
        logging.error(f"Lỗi trong quá trình chuyển đổi: {str(e)}")
        raise

In [ ]:
""""" MAIN - CONVERT TO FAISS """""
if os.path.exists(faiss_path):
    print(f"\nFaiss loaded from {faiss_path}\n")
else:
    if os.path.exists(torch_path):
        convert_pt_to_faiss(torch_path=torch_path, faiss_path=faiss_path, mapping_path=mapping_path, data_path=mapping_data, data_key = DATA_KEY, nlist = 100, use_pickle = False)
    else:
        print(f"TORCH path does not exist: {torch_path}")